In [29]:
%pip install langchain-community langchain-core
%pip install sklearn
%pip install pandas
%pip install nltk


You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/myenv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [30]:
import os
import json
import rdflib
import random
from typing import List
from openai import OpenAI
from dotenv import load_dotenv
from langchain_community.graphs import OntotextGraphDBGraph
from constants import (
    TRAINED_MODEL_ID_PATH,
    TEST_DATA_PATH,
    SPARQL_FIX_PROMPT,
    QA_PROMPT,
    SPARQL_GENERATION_PROMPT,
)
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from nltk.translate.bleu_score import sentence_bleu
import pandas as pd
from tqdm import tqdm

In [31]:
class CustomOntotextGraphDBGraph(OntotextGraphDBGraph):
    def query(
        self,
        query: str,
    ) -> List[rdflib.query.ResultRow]:
        """
        Query the graph.
        """
        from rdflib.query import ResultRow

        res = self.graph.query(query)
        if res.type == "ASK":
            return [r for r in res if isinstance(r, bool)]
        return [r for r in res if isinstance(r, ResultRow)]

## Create Graph OBJ


In [32]:
config = {
    "query_endpoint": "http://localhost:7200/repositories/imkg",
    "local_file": "/Users/jerry/Desktop/FYP-working/fine-tune-openai-KGQA/KG/iMKG.ttl",
    "local_file_format": "turtle",
}
try:
    graph = CustomOntotextGraphDBGraph(**config)
    print("Graph object created successfully.")
except Exception as e:
    print("Failed to create graph object:", e)

Graph object created successfully.


In [33]:
load_dotenv()

True

In [34]:
client = OpenAI()
with open(TRAINED_MODEL_ID_PATH, "r", encoding="utf-8") as f:
    model_name = f.read()
f.close()
print(f"Model name: {model_name}")

Model name: ft:gpt-4o-2024-08-06:personal::AyvemhXl


In [35]:
TEST_DATA_PATH = os.path.join(os.getcwd(), "data", "qa_test.json")
with open(TEST_DATA_PATH, "r", encoding="utf-8") as f:
    test_dataset = json.load(f)

test_data_point = random.sample(test_dataset, 1)[0]
print(f"Test data point:\n{json.dumps(test_data_point, indent=4)}")

Test data point:
{
    "qid": "mcqa-9279b7bd-e67e-11ee-b2b4-58961d663d9c",
    "question": "Can you tell me which actor was in both [One-Trick Pony] and the film [The Beastmaster]?",
    "question_type": "movie_to_actor_constraint_movie",
    "topic_entity": [
        "One-Trick Pony",
        "The Beastmaster"
    ],
    "topic_entity_id": [
        "https://www.wikidata.org/entity/Q4045814",
        "https://www.wikidata.org/entity/Q813050"
    ],
    "answer": [
        "Rip Torn"
    ],
    "answer_id": [
        "https://www.wikidata.org/entity/Q313522"
    ],
    "sparql": "PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?label ?uri WHERE { ?e1 ?rel ?uri; rdfs:label \"One-Trick Pony\" . ?e3 ?rel ?uri; rdfs:label \"The Beastmaster\" . ?rel rdfs:label \"starred_actors\" . ?uri rdfs:label ?label . }"
}


## Call OPENAI FINETUNE MODEL


In [47]:
def call_openai(messages: list, model_name, **kwargs) -> str:
    print(f"Messages: {messages}")
    print(f"Model: {model_name}")
    response = client.chat.completions.create(
        messages=messages, model=model_name, **kwargs
    )

    return response.choices[0].message.content

In [ ]:
query_generation_messages = [
    {"role": "system", "content": SPARQL_GENERATION_PROMPT},
    {"role": "user", "content": f"{test_data_point['question']}"},
]
query = call_openai(
    messages=query_generation_messages, model_name=model_name, temperature=0, top_p=0
)
print(f"Generated SparQL:\n{query}")
# add generated query into data point ban dau

Messages: [{'role': 'system', 'content': '\nYou are a useful SparQL assistant. You are tasked to review a question and generate a SparQL to answer the question.\nSparQL Database used is WikiData. [<text>] is topic entity in the question.\nOnly use these two prefixes PREFIX wd: <https://www.wikidata.org/entity/> and PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> if needed.\nDo not use wdt syntax to query WikiData\n'}, {'role': 'user', 'content': 'Can you tell me which actor was in both [One-Trick Pony] and the film [The Beastmaster]?'}]
Model: ft:gpt-4o-2024-08-06:personal::AyvemhXl
Generated SparQL:
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?label ?uri WHERE { ?e1 ?rel ?uri; rdfs:label "One-Trick Pony" . ?e3 ?rel ?uri; rdfs:label "The Beastmaster" . ?rel rdfs:label "starred_actors" . ?uri rdfs:label ?label . }


# Dump answer results


In [51]:
TEST_DATA_PATH = os.path.join(os.getcwd(), "data", "qa_test.json")
TEST_DATA_RESULTS_PATH = os.path.join(os.getcwd(), "data", "Gpt4o-qa_test_results.json")

with open(TEST_DATA_PATH, "r", encoding="utf-8") as f:
    test_dataset = json.load(f)[:100]

if os.path.exists(TEST_DATA_RESULTS_PATH):
    with open(TEST_DATA_RESULTS_PATH, "r", encoding="utf-8") as f:
        test_results = json.load(f)
else:
    test_results = []

print(test_dataset)

[{'qid': 'mcqa-9274ab0a-e67e-11ee-88b8-58961d663d9c', 'question': 'which actor starred in the movies written by [Susan Hill] and was also in [The Woman in Black]?', 'question_type': 'writer_to_movie_to_actor_constraint_movie', 'topic_entity': ['Susan Hill', 'The Woman in Black'], 'topic_entity_id': ['https://www.wikidata.org/entity/Q269982', 'https://www.wikidata.org/entity/Q21185894', 'https://www.wikidata.org/entity/Q3490770', 'https://www.wikidata.org/entity/Q65044097', 'https://www.wikidata.org/entity/Q841274'], 'answer': ['Adrian Rawlins', 'Trevor Cooper', 'Peter Guinness', 'David Ryall', 'Pauline Moran', 'Steven Mackintosh', 'John Cater', 'Andy Nyman', 'Clare Holman', 'David Daker', 'Bernard Hepton', 'Caroline John', 'Ciarán Hinds', 'Daniel Radcliffe', 'Shaun Dooley', 'Janet McTeer', 'Roger Allam', 'Liz White', 'Jessica Raine', 'Sophie Stuckey', 'David Burke'], 'answer_id': ['https://www.wikidata.org/entity/Q374041', 'https://www.wikidata.org/entity/Q7839105', 'https://www.wikida

## HANDLE ERROR (IF THERE IS)


In [28]:
NO_RETRRY = 3
query_results = None
try:
    query_results = graph.query(query)
    print(f"Query results: {query_results}")
except Exception as e:
    retries = 0
    error_message = str(e)
    while retries < NO_RETRRY:
        try:
            print(f"Invalid query. Retry count: {retries+1}")
            fix_sparql_query_messages = [
                {"role": "system", "content": SPARQL_FIX_PROMPT},
                {
                    "role": "user",
                    "content": f"""Question: {test_data_point['question']}
Previous SparQL query: {query}
Error message: {error_message}
""",
                },
            ]
            query = call_openai(
                messages=fix_sparql_query_messages,
                model_name=model_name,
                temperature=0,
                top_p=0,
            )
            print(f"New query: {query}")
            query_results = graph.query(query)
            print(f"Query results: {query_results}")
            break
        except Exception as e:
            retries += 1
            error_message = str(e)
if query_results == None:
    raise Exception("Invalid SparQL query. Stop answering")

Query results: [(rdflib.term.Literal('The Krays'), rdflib.term.URIRef('https://www.wikidata.org/entity/Q12125426')), (rdflib.term.Literal('Sex and Lies in Sin City'), rdflib.term.URIRef('https://www.wikidata.org/entity/Q2275467')), (rdflib.term.Literal('Romeo Is Bleeding'), rdflib.term.URIRef('https://www.wikidata.org/entity/Q624857')), (rdflib.term.Literal('Dating Game Killer'), rdflib.term.URIRef('https://www.wikidata.org/entity/Q73538437'))]


In [24]:
qa_answering_prompt = QA_PROMPT.replace("{{context}}", str(query_results))
qa_answering_prompt = qa_answering_prompt.replace("{{query}}", str(query))
qa_answering_messages = [
    {"role": "system", "content": qa_answering_prompt},
    {"role": "user", "content": f"Question: {test_data_point['question']}"},
]
final_response = call_openai(
    messages=qa_answering_messages, model_name="gpt-4o", temperature=0.5
)
print(f"Final Response:\n{final_response}")

Messages: [{'role': 'system', 'content': 'Generate a natural language response from the results of a SPARQL query.\nDon\'t use any internal knowledge to answer the question,\nJust say you don\'t know if no information is available from The results of a SPARQL query.\nIf the question is Yes/No question and there is no information available, answer No.\nSparQL query: PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT DISTINCT ?label ?uri WHERE { ?rel1 rdfs:label "directed_by" . ?e1 ?rel1 ?e2 ; rdfs:label "Let Him Have It" . ?uri ?rel1 ?e2 ; ?rel4 ?e4 ; rdfs:label ?label . FILTER (?label != "Let Him Have It") ?rel4 rdfs:label "has_genre" . ?e4 rdfs:label "crime film" . }\nThe results of a SparQL query: [(rdflib.term.Literal(\'The Krays\'), rdflib.term.URIRef(\'https://www.wikidata.org/entity/Q12125426\')), (rdflib.term.Literal(\'Sex and Lies in Sin City\'), rdflib.term.URIRef(\'https://www.wikidata.org/entity/Q2275467\')), (rdflib.term.Literal(\'Romeo Is Bleeding\'), rdflib.term.

# Calculate metrics


In [25]:
# our query
generated_results = set(graph.query(query))

# sample query
sample_results = set(graph.query(test_data_point["sparql"]))

# Compare the results
if generated_results == sample_results:
    print("The queries produce identical results.")
else:
    print("The queries produce different results.")

The queries produce identical results.
